# Day11_1: 텍스트 분석 기법 (Text Analysis Techniques)

## 📚 학습 목표

**Part 1: 기초**
1. 단어 빈도(Counter)로 텍스트 분석 시작하기
2. n-gram 개념 이해하고 활용하기
3. CountVectorizer로 문서-단어 행렬 만들기
4. TF-IDF 개념 이해하기
5. TfidfVectorizer로 키워드 중요도 추출하기

**Part 2: 심화**
1. 코사인 유사도로 문서 비교하기
2. 자카드 유사도 이해하기
3. 중복 문서 탐지하기
4. 간단한 문서 검색 시스템 구현하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| TF-IDF | 문서별 핵심 키워드 추출 | 뉴스 기사 자동 태깅, SEO 최적화 |
| n-gram | 연속된 단어 패턴 분석 | 제품 리뷰에서 "배송이 빠르다" 탐지 |
| 코사인 유사도 | 유사 문서 찾기 | 중복 기사 탐지, 추천 시스템 |
| 문서 벡터화 | 텍스트를 숫자로 변환 | ML 모델 입력 데이터 생성 |

**분석가 관점**: 텍스트는 정형화되지 않은 데이터지만, 벡터화와 유사도 측정을 통해 패턴을 발견하고 분류할 수 있습니다.

---

# Part 1: 기초

---

## 1.1 단어 빈도 분석 (Word Frequency)

### Counter를 활용한 기본 빈도 분석

텍스트 분석의 첫걸음은 단어 빈도를 세는 것입니다.

In [1]:
# 필요한 라이브러리 설치 확인
# !pip install scikit-learn plotly pandas

from collections import Counter
import pandas as pd

# 예시 고객 리뷰 데이터
reviews = [
    "배송이 빠르고 상품 품질이 좋아요",
    "배송이 너무 빠르고 포장도 깔끔해요",
    "상품 품질이 좋아요 다시 구매할게요",
    "배송이 빠르고 서비스가 좋아요"
]

# 모든 리뷰를 하나로 합치고 단어 분리
all_words = []
for review in reviews:
    words = review.split()
    all_words.extend(words)

# 단어 빈도 계산
word_counts = Counter(all_words)
print("단어 빈도:")
for word, count in word_counts.most_common(10):
    print(f"  {word}: {count}회")

단어 빈도:
  배송이: 3회
  빠르고: 3회
  좋아요: 3회
  상품: 2회
  품질이: 2회
  너무: 1회
  포장도: 1회
  깔끔해요: 1회
  다시: 1회
  구매할게요: 1회


In [2]:
# Plotly로 상위 키워드 시각화
import plotly.express as px

# 상위 10개 단어
top_words = word_counts.most_common(10)
df_words = pd.DataFrame(top_words, columns=['단어', '빈도'])

fig = px.bar(
    df_words, 
    x='빈도', 
    y='단어', 
    orientation='h',
    title='상위 10개 빈출 단어',
    color='빈도',
    color_continuous_scale='Blues'
)
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

### 💡 실무 예시: 불만 키워드 탐지

고객센터에 접수된 불만 사항에서 자주 언급되는 키워드를 찾아봅시다.

In [3]:
# 고객 불만 데이터
complaints = [
    "환불 요청했는데 처리가 너무 느려요",
    "상품이 파손되어 배송됐어요 환불 해주세요",
    "교환 신청했는데 연락이 없어요",
    "환불 처리가 안 되고 있어요",
    "배송 지연이 너무 심해요 취소하고 싶어요"
]

# 불만 키워드 추출
complaint_words = []
for c in complaints:
    complaint_words.extend(c.split())

complaint_counts = Counter(complaint_words)
print("불만 키워드 TOP 5:")
for word, count in complaint_counts.most_common(5):
    print(f"  '{word}': {count}회 언급")

불만 키워드 TOP 5:
  '환불': 3회 언급
  '처리가': 2회 언급
  '너무': 2회 언급
  '요청했는데': 1회 언급
  '느려요': 1회 언급


---

## 1.2 n-gram 분석

### n-gram이란?

- **unigram (1-gram)**: 단일 단어 ("배송", "빠르다")
- **bigram (2-gram)**: 연속된 2개 단어 ("배송이 빠르다")
- **trigram (3-gram)**: 연속된 3개 단어 ("정말 배송이 빠르다")

n-gram을 사용하면 단어의 **맥락**을 파악할 수 있습니다.

In [ ]:
# n-gram 생성 함수
def get_ngrams(text, n):
    """텍스트에서 n-gram 추출"""
    words = text.split()
    ngrams = []
    for i in range(len(words) - n + 1):
        ngram = ' '.join(words[i:i+n])
        ngrams.append(ngram)
    return ngrams

# 예시 텍스트
text = "배송이 빠르고 상품 품질이 좋아요"

print("원본:", text)
print("\n1-gram (unigram):", get_ngrams(text, 1))
print("2-gram (bigram):", get_ngrams(text, 2))
print("3-gram (trigram):", get_ngrams(text, 3))

In [ ]:
# 여러 문서에서 bigram 빈도 분석
reviews = [
    "배송이 빠르고 포장이 깔끔해요",
    "배송이 빠르고 상품이 좋아요",
    "상품이 좋아요 추천합니다",
    "포장이 깔끔해요 감사합니다"
]

# 모든 bigram 수집
all_bigrams = []
for review in reviews:
    all_bigrams.extend(get_ngrams(review, 2))

bigram_counts = Counter(all_bigrams)
print("자주 등장하는 bigram:")
for bigram, count in bigram_counts.most_common(5):
    print(f"  '{bigram}': {count}회")

### 💡 실무 예시: 제품 리뷰에서 핵심 표현 찾기

bigram 분석으로 고객이 자주 사용하는 표현 패턴을 발견합니다.

In [ ]:
# 전자제품 리뷰 데이터
product_reviews = [
    "가성비 좋고 디자인이 예뻐요",
    "가성비 최고 강력 추천합니다",
    "배터리 오래가고 가성비 좋아요",
    "강력 추천 가성비 최고예요"
]

# bigram 추출
bigrams = []
for r in product_reviews:
    bigrams.extend(get_ngrams(r, 2))

# 빈도 시각화
bigram_df = pd.DataFrame(
    Counter(bigrams).most_common(8), 
    columns=['표현', '빈도']
)

fig = px.bar(
    bigram_df,
    x='빈도',
    y='표현',
    orientation='h',
    title='자주 사용되는 2단어 표현 (Bigram)',
    color='빈도'
)
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

---

## 1.3 CountVectorizer: 문서-단어 행렬

### 문서를 숫자 벡터로 변환하기

`CountVectorizer`는 텍스트를 **Bag of Words** 모델로 변환합니다.

- 각 문서를 단어 빈도 벡터로 표현
- 머신러닝 모델의 입력으로 사용 가능

In [4]:
from sklearn.feature_extraction.text import CountVectorizer         #fit은 훈련하는거 모듈
                                                                    #transform은 변형, 처리 모듈
# 샘플 문서                                                          #predict 예측
documents = [
    "배송이 빠르고 상품이 좋아요",
    "상품 품질이 좋아요 추천합니다",
    "배송이 느리고 상품이 파손됐어요",
    "빠른 배송 감사합니다 추천합니다"
]

# CountVectorizer 생성
vectorizer = CountVectorizer()

# 문서를 벡터로 변환
count_matrix = vectorizer.fit_transform(documents)

# 결과 확인
print("단어 목록:", vectorizer.get_feature_names_out())
print("\n문서-단어 행렬:")
print(count_matrix.toarray())

단어 목록: ['감사합니다' '느리고' '배송' '배송이' '빠르고' '빠른' '상품' '상품이' '좋아요' '추천합니다' '파손됐어요'
 '품질이']

문서-단어 행렬:
[[0 0 0 1 1 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 1 0 1 1 0 1]
 [0 1 0 1 0 0 0 1 0 0 1 0]
 [1 0 1 0 0 1 0 0 0 1 0 0]]


In [ ]:
# DataFrame으로 보기 좋게 표시
df_counts = pd.DataFrame(
    count_matrix.toarray(),
    columns=vectorizer.get_feature_names_out(),
    index=[f'문서{i+1}' for i in range(len(documents))]
)

print("문서-단어 행렬 (Document-Term Matrix):")
df_counts

### CountVectorizer 주요 옵션

| 옵션 | 설명 | 예시 |
|------|------|------|
| `max_features` | 최대 단어 수 제한 | `max_features=100` |
| `min_df` | 최소 문서 빈도 | `min_df=2` (2개 이상 문서) |
| `max_df` | 최대 문서 빈도 | `max_df=0.8` (80% 이하) |
| `ngram_range` | n-gram 범위 | `ngram_range=(1, 2)` |
| `stop_words` | 불용어 제거 | `stop_words='english'` |

In [ ]:
# n-gram 포함 CountVectorizer
vectorizer_ngram = CountVectorizer(ngram_range=(1, 2))  # unigram + bigram

count_matrix_ngram = vectorizer_ngram.fit_transform(documents)

print("unigram + bigram 단어 목록:")
print(vectorizer_ngram.get_feature_names_out())

---

## 1.4 TF-IDF 개념 이해

### TF-IDF란?

**TF-IDF** (Term Frequency - Inverse Document Frequency)는 단어의 **중요도**를 측정합니다.

- **TF (Term Frequency)**: 문서 내 단어 빈도 (높을수록 중요)
- **IDF (Inverse Document Frequency)**: 전체 문서에서의 희소성 (드물수록 중요)

**TF-IDF = TF x IDF**

- 문서 내에서 자주 등장하지만, 전체 문서에서는 드문 단어 = **높은 TF-IDF**
- "은", "는", "이" 같은 일반적 단어 = **낮은 TF-IDF**

In [5]:
# TF-IDF 직관적 이해
print("=== TF-IDF 직관적 예시 ===")
print()
print("문서1: '파이썬 프로그래밍 학습'")
print("문서2: '자바 프로그래밍 학습'")
print("문서3: '데이터 분석 파이썬'")
print()
print("'프로그래밍': 모든 문서에 등장 -> IDF 낮음 -> TF-IDF 낮음")
print("'파이썬': 일부 문서에만 등장 -> IDF 높음 -> TF-IDF 높음")
print()
print("=> '파이썬'이 문서를 구별하는 더 중요한 키워드!")

=== TF-IDF 직관적 예시 ===

문서1: '파이썬 프로그래밍 학습'
문서2: '자바 프로그래밍 학습'
문서3: '데이터 분석 파이썬'

'프로그래밍': 모든 문서에 등장 -> IDF 낮음 -> TF-IDF 낮음
'파이썬': 일부 문서에만 등장 -> IDF 높음 -> TF-IDF 높음

=> '파이썬'이 문서를 구별하는 더 중요한 키워드!


---

## 1.5 TfidfVectorizer로 키워드 추출

### 기본 사용법

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 뉴스 기사 샘플
news_articles = [
    "삼성전자 반도체 수출 증가 반도체 시장 호황",
    "현대자동차 전기차 판매 증가 전기차 시장 성장",
    "네이버 AI 기술 투자 AI 서비스 확대",
    "카카오 AI 플랫폼 출시 AI 신사업 추진"
]

# TfidfVectorizer 생성 및 변환
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(news_articles)

# 결과를 DataFrame으로
df_tfidf = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=tfidf_vectorizer.get_feature_names_out(),
    index=['삼성 기사', '현대 기사', '네이버 기사', '카카오 기사']
)

print("TF-IDF 행렬:")
df_tfidf.round(3)

TF-IDF 행렬:


,ai,기술,네이버,반도체,삼성전자,서비스,성장,수출,시장,신사업,...,증가,추진,출시,카카오,투자,판매,플랫폼,현대자동차,호황,확대
삼성 기사,0.000,0.000,0.000,0.697,0.348,0.000,0.000,0.348,0.275,0.000,...,0.275,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.348,0.000
현대 기사,0.000,0.000,0.000,0.000,0.000,0.000,0.348,0.000,0.275,0.000,...,0.275,0.000,0.000,0.000,0.000,0.348,0.000,0.348,0.000,0.000
네이버 기사,0.576,0.365,0.365,0.000,0.000,0.365,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.365,0.000,0.000,0.000,0.000,0.365
카카오 기사,0.576,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.365,...,0.000,0.365,0.365,0.365,0.000,0.000,0.365,0.000,0.000,0.000


In [7]:
# 각 문서별 중요 키워드 추출 함수
def get_top_keywords(tfidf_matrix, feature_names, doc_index, top_n=5):
    """문서에서 TF-IDF 상위 키워드 추출"""
    # 해당 문서의 TF-IDF 점수
    tfidf_scores = tfidf_matrix[doc_index].toarray().flatten()
    
    # 점수와 단어를 묶어서 정렬
    word_scores = list(zip(feature_names, tfidf_scores))
    word_scores_sorted = sorted(word_scores, key=lambda x: x[1], reverse=True)
    
    return word_scores_sorted[:top_n]

# 각 기사별 핵심 키워드
feature_names = tfidf_vectorizer.get_feature_names_out()

for i, article_name in enumerate(['삼성 기사', '현대 기사', '네이버 기사', '카카오 기사']):
    keywords = get_top_keywords(tfidf_matrix, feature_names, i, top_n=3)
    print(f"\n{article_name} 핵심 키워드:")
    for word, score in keywords:
        print(f"  {word}: {score:.3f}")


삼성 기사 핵심 키워드:
  반도체: 0.697
  삼성전자: 0.348
  수출: 0.348

현대 기사 핵심 키워드:
  전기차: 0.697
  성장: 0.348
  판매: 0.348

네이버 기사 핵심 키워드:
  ai: 0.576
  기술: 0.365
  네이버: 0.365

카카오 기사 핵심 키워드:
  ai: 0.576
  신사업: 0.365
  추진: 0.365


### 💡 실무 예시: 고객 리뷰 키워드 추출

In [ ]:
# 카테고리별 리뷰 데이터
category_reviews = {
    "전자제품": "가성비 좋고 배터리 오래가요 화면이 선명해요",
    "의류": "재질이 좋고 사이즈가 정확해요 색상이 예뻐요",
    "식품": "맛있고 신선해요 포장이 꼼꼼해요 유통기한 넉넉해요",
    "가구": "조립이 쉽고 튼튼해요 디자인이 깔끔해요"
}

# TF-IDF 분석
reviews_list = list(category_reviews.values())
categories = list(category_reviews.keys())

tfidf = TfidfVectorizer()
tfidf_result = tfidf.fit_transform(reviews_list)        #sparse한 데이터가 나올거임
features = tfidf.get_feature_names_out()            #단어가 나올 거임

# 카테고리별 대표 키워드
print("=== 카테고리별 핵심 키워드 ===")
for i, category in enumerate(categories):
    top_keywords = get_top_keywords(tfidf_result, features, i, top_n=3)
    keywords_str = ', '.join([f"{w}({s:.2f})" for w, s in top_keywords])
    print(f"{category}: {keywords_str}")

=== 카테고리별 핵심 키워드 ===
전자제품: 가성비(0.42), 배터리(0.42), 선명해요(0.42)
의류: 사이즈가(0.42), 색상이(0.42), 예뻐요(0.42)
식품: 꼼꼼해요(0.41), 넉넉해요(0.41), 맛있고(0.41)
가구: 깔끔해요(0.45), 디자인이(0.45), 쉽고(0.45)


---

# Part 2: 심화

---

## 2.1 코사인 유사도 (Cosine Similarity)

### 문서 간 유사도 측정

코사인 유사도는 두 벡터 간의 **각도**를 측정합니다.

- **값 범위**: 0 (완전히 다름) ~ 1 (완전히 같음)
- **장점**: 문서 길이에 영향 받지 않음

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

# 비교할 문서들
documents = [
    "파이썬 머신러닝 데이터 분석",  # 문서1
    "파이썬 데이터 분석 시각화",    # 문서2 (문서1과 유사)
    "자바 웹 개발 스프링",          # 문서3 (문서1과 다름)
    "데이터 분석 머신러닝 딥러닝"    # 문서4 (문서1과 유사)
]

# TF-IDF 벡터화
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(documents)

# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix)

# 결과를 DataFrame으로
df_similarity = pd.DataFrame(
    cosine_sim,
    index=[f'문서{i+1}' for i in range(len(documents))],
    columns=[f'문서{i+1}' for i in range(len(documents))]
)

print("문서 간 코사인 유사도:")
df_similarity.round(3)

문서 간 코사인 유사도:


,문서1,문서2,문서3,문서4
문서1,1.000,0.641,0.0,0.641
문서2,0.641,1.000,0.0,0.334
문서3,0.000,0.000,1.0,0.000
문서4,0.641,0.334,0.0,1.000


In [10]:
# 히트맵으로 시각화
import plotly.figure_factory as ff

# 문서 라벨
labels = [f'문서{i+1}' for i in range(len(documents))]

fig = ff.create_annotated_heatmap(
    z=cosine_sim.round(2),
    x=labels,
    y=labels,
    colorscale='Blues',
    showscale=True
)

fig.update_layout(
    title='문서 간 코사인 유사도 히트맵',
    xaxis_title='문서',
    yaxis_title='문서'
)

fig.show()

### 특정 문서와 가장 유사한 문서 찾기

In [12]:
def find_similar_documents(query_index, similarity_matrix, documents, top_n=3):
    """주어진 문서와 가장 유사한 문서 찾기"""
    # 유사도 점수
    sim_scores = list(enumerate(similarity_matrix[query_index]))
    
    # 자기 자신 제외하고 정렬
    sim_scores = [(i, score) for i, score in sim_scores if i != query_index]
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 상위 n개 반환
    results = []
    for idx, score in sim_scores[:top_n]:
        results.append({
            '문서': documents[idx],
            '유사도': round(score, 3)
        })
    
    return results

# 문서1과 유사한 문서 찾기
print(f"기준 문서: '{documents[0]}'")
print("\n유사한 문서:")
similar_docs = find_similar_documents(0, cosine_sim, documents)
for doc in similar_docs:
    print(f"  유사도 {doc['유사도']}: {doc['문서']}")

기준 문서: '파이썬 머신러닝 데이터 분석'

유사한 문서:
  유사도 0.641: 파이썬 데이터 분석 시각화
  유사도 0.641: 데이터 분석 머신러닝 딥러닝
  유사도 0.0: 자바 웹 개발 스프링


### 💡 실무 예시: 뉴스 기사 추천 시스템

In [ ]:
# 뉴스 기사 데이터
news_db = [
    {"id": 1, "title": "삼성전자 반도체 수출 역대 최고", "category": "경제"},
    {"id": 2, "title": "SK하이닉스 반도체 투자 확대", "category": "경제"},
    {"id": 3, "title": "손흥민 토트넘 결승골 활약", "category": "스포츠"},
    {"id": 4, "title": "TSMC 반도체 생산량 증가", "category": "경제"},
    {"id": 5, "title": "이강인 PSG 리그 우승 기여", "category": "스포츠"}
]

# 기사 제목으로 TF-IDF
titles = [n["title"] for n in news_db]
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(titles)
cosine_sim = cosine_similarity(tfidf_matrix)

# 사용자가 읽은 기사 (id=1)
user_article_idx = 0
print(f"읽은 기사: {news_db[user_article_idx]['title']}")
print("\n추천 기사:")

similar = find_similar_documents(user_article_idx, cosine_sim, titles)
for doc in similar:
    print(f"  - {doc['문서']} (유사도: {doc['유사도']})")

---

## 2.2 자카드 유사도 (Jaccard Similarity)

### 집합 기반 유사도

자카드 유사도는 두 집합의 **교집합/합집합** 비율을 계산합니다.

$$\text{Jaccard}(A, B) = \frac{|A \cap B|}{|A \cup B|}$$

- **값 범위**: 0 (공통 없음) ~ 1 (완전히 같음)
- **장점**: 단어 빈도 무시, 단순 존재 여부만 확인

In [ ]:
def jaccard_similarity(set1, set2):
    """두 집합의 자카드 유사도 계산"""
    intersection = len(set1 & set2)  # 교집합
    union = len(set1 | set2)         # 합집합
    
    if union == 0:
        return 0.0
    return intersection / union

# 두 문서 비교
doc1 = "파이썬 머신러닝 데이터 분석"
doc2 = "파이썬 데이터 분석 시각화"

# 단어 집합으로 변환
set1 = set(doc1.split())
set2 = set(doc2.split())

print(f"문서1 단어: {set1}")
print(f"문서2 단어: {set2}")
print(f"\n교집합: {set1 & set2}")
print(f"합집합: {set1 | set2}")
print(f"\n자카드 유사도: {jaccard_similarity(set1, set2):.3f}")

In [ ]:
# 여러 문서 비교
documents = [
    "파이썬 머신러닝 데이터 분석",
    "파이썬 데이터 분석 시각화",
    "자바 웹 개발 스프링",
    "파이썬 웹 크롤링 스크래핑"
]

# 자카드 유사도 행렬 계산
n = len(documents)
jaccard_matrix = []

for i in range(n):
    row = []
    set_i = set(documents[i].split())
    for j in range(n):
        set_j = set(documents[j].split())
        sim = jaccard_similarity(set_i, set_j)
        row.append(sim)
    jaccard_matrix.append(row)

# DataFrame으로 표시
df_jaccard = pd.DataFrame(
    jaccard_matrix,
    index=[f'문서{i+1}' for i in range(n)],
    columns=[f'문서{i+1}' for i in range(n)]
)

print("자카드 유사도 행렬:")
df_jaccard.round(3)

### 코사인 vs 자카드 비교

| 특성 | 코사인 유사도 | 자카드 유사도 |
|------|-------------|-------------|
| 기준 | 벡터 각도 | 집합 중복 |
| 빈도 고려 | O (TF-IDF) | X (존재 여부만) |
| 적합한 경우 | 긴 문서 비교 | 짧은 텍스트, 태그 비교 |
| 예시 | 뉴스 기사 유사도 | 해시태그 유사도 |

---

## 2.3 중복 문서 탐지

### 유사도 임계값으로 중복 찾기

In [ ]:
def find_duplicates(documents, threshold=0.7):
    """유사도가 임계값 이상인 문서 쌍 찾기"""
    # TF-IDF 벡터화
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(documents)
    
    # 코사인 유사도 계산
    cosine_sim = cosine_similarity(tfidf_matrix)
    
    # 중복 쌍 찾기
    duplicates = []
    n = len(documents)
    for i in range(n):
        for j in range(i + 1, n):  # 상삼각 행렬만 확인
            if cosine_sim[i][j] >= threshold:
                duplicates.append({
                    '문서1_idx': i,
                    '문서2_idx': j,
                    '문서1': documents[i],
                    '문서2': documents[j],
                    '유사도': round(cosine_sim[i][j], 3)
                })
    
    return duplicates

# 테스트 데이터 (일부 중복 포함)
news_headlines = [
    "삼성전자 반도체 수출 역대 최고 실적 달성",
    "삼성전자 반도체 수출 사상 최대 기록",  # 중복 의심
    "현대차 전기차 판매량 증가",
    "LG전자 가전 사업 호조",
    "삼성 반도체 수출 역대 최고치 경신"  # 중복 의심
]

# 중복 탐지 (임계값 0.5)
duplicates = find_duplicates(news_headlines, threshold=0.5)

print("=== 중복 의심 문서 ===\n")
for dup in duplicates:
    print(f"유사도: {dup['유사도']}")
    print(f"  문서1: {dup['문서1']}")
    print(f"  문서2: {dup['문서2']}")
    print()

### 💡 실무 예시: 중복 리뷰 탐지 시스템

In [ ]:
# 쇼핑몰 리뷰 데이터 (일부 조작 리뷰 포함)
reviews = [
    {"id": 1, "user": "user1", "text": "배송 빠르고 상품 좋아요 강력 추천"},
    {"id": 2, "user": "user2", "text": "상품 품질 최고입니다 감사합니다"},
    {"id": 3, "user": "user3", "text": "배송 빠르고 상품 좋아요 추천합니다"},  # user1과 유사
    {"id": 4, "user": "user4", "text": "가격 대비 만족스러워요"},
    {"id": 5, "user": "user5", "text": "배송이 빠르고 상품이 좋아요 추천"} # user1과 유사
]

# 중복 탐지
review_texts = [r["text"] for r in reviews]
duplicates = find_duplicates(review_texts, threshold=0.6)

print("=== 조작 의심 리뷰 탐지 ===")
print(f"총 리뷰: {len(reviews)}개\n")

for dup in duplicates:
    idx1, idx2 = dup['문서1_idx'], dup['문서2_idx']
    print(f"유사도 {dup['유사도']} - 조작 의심!")
    print(f"  리뷰 {reviews[idx1]['id']} ({reviews[idx1]['user']}): {reviews[idx1]['text']}")
    print(f"  리뷰 {reviews[idx2]['id']} ({reviews[idx2]['user']}): {reviews[idx2]['text']}")
    print()

---

## 2.4 간단한 문서 검색 시스템

### TF-IDF 기반 검색 엔진

In [13]:
class SimpleSearchEngine:       #class엔진 
    """TF-IDF 기반 간단한 검색 엔진"""
    
    def __init__(self, documents):
        self.documents = documents
        self.vectorizer = TfidfVectorizer()
        self.tfidf_matrix = self.vectorizer.fit_transform(documents)
    
    def search(self, query, top_n=3):
        """검색 쿼리로 관련 문서 찾기"""
        # 쿼리를 벡터로 변환
        query_vector = self.vectorizer.transform([query])
        
        # 모든 문서와 유사도 계산
        similarities = cosine_similarity(query_vector, self.tfidf_matrix).flatten()
        
        # 상위 n개 결과
        top_indices = similarities.argsort()[::-1][:top_n]
        
        results = []
        for idx in top_indices:
            if similarities[idx] > 0:  # 유사도 0보다 큰 것만
                results.append({
                    '문서': self.documents[idx],
                    '유사도': round(similarities[idx], 3)
                })
        
        return results

In [14]:
# 문서 데이터베이스
document_db = [
    "파이썬 머신러닝 입문 강좌입니다",
    "데이터 분석을 위한 판다스 튜토리얼",
    "딥러닝 텐서플로우 기초 가이드",
    "파이썬 웹 크롤링 실습 예제",
    "머신러닝 알고리즘 비교 분석",
    "자연어 처리 NLP 기초 강좌",
    "파이썬 데이터 시각화 플롯리"
]

# 검색 엔진 생성
search_engine = SimpleSearchEngine(document_db)

# 검색 테스트
queries = ["머신러닝", "데이터 분석", "파이썬 기초"]

for query in queries:
    print(f"\n검색어: '{query}'")
    results = search_engine.search(query, top_n=3)
    for i, result in enumerate(results, 1):
        print(f"  {i}. [{result['유사도']}] {result['문서']}")


검색어: '머신러닝'
  1. [0.465] 파이썬 머신러닝 입문 강좌입니다
  2. [0.432] 머신러닝 알고리즘 비교 분석

검색어: '데이터 분석'
  1. [0.401] 머신러닝 알고리즘 비교 분석
  2. [0.297] 파이썬 데이터 시각화 플롯리
  3. [0.245] 데이터 분석을 위한 판다스 튜토리얼

검색어: '파이썬 기초'
  1. [0.329] 딥러닝 텐서플로우 기초 가이드
  2. [0.291] 자연어 처리 NLP 기초 강좌
  3. [0.258] 파이썬 데이터 시각화 플롯리


### 💡 실무 예시: FAQ 자동 응답 시스템

In [15]:
# FAQ 데이터베이스
faq_db = [
    {"q": "배송 얼마나 걸리나요", "a": "일반 배송은 2-3일, 빠른 배송은 1일 소요됩니다."},
    {"q": "환불 가능한가요", "a": "구매 후 7일 이내 환불 가능합니다."},
    {"q": "교환 신청 어떻게 하나요", "a": "마이페이지 > 주문내역에서 교환 신청하세요."},
    {"q": "결제 방법 뭐가 있나요", "a": "카드, 계좌이체, 간편결제 모두 가능합니다."},
    {"q": "포인트 어떻게 사용하나요", "a": "결제 시 포인트 사용 선택하시면 됩니다."}
]

# FAQ 검색 엔진
faq_questions = [f["q"] for f in faq_db]
faq_engine = SimpleSearchEngine(faq_questions)

def answer_question(user_question):
    """사용자 질문에 가장 적합한 FAQ 답변 찾기"""
    results = faq_engine.search(user_question, top_n=1)
    
    if results and results[0]['유사도'] > 0.2:
        # 가장 유사한 FAQ 찾기
        matched_q = results[0]['문서']
        for faq in faq_db:
            if faq['q'] == matched_q:
                return faq['a'], results[0]['유사도']
    
    return "죄송합니다. 해당 질문에 대한 답변을 찾을 수 없습니다.", 0.0

# 테스트
test_questions = [
    "배송 시간이 얼마나 되나요?",
    "환불하고 싶어요",
    "결제는 어떻게 해요?",
    "제품 AS 문의"
]

print("=== FAQ 자동 응답 시스템 ===")
for q in test_questions:
    answer, score = answer_question(q)
    print(f"\nQ: {q}")
    print(f"A: {answer}")
    print(f"   (신뢰도: {score:.2f})")

=== FAQ 자동 응답 시스템 ===

Q: 배송 시간이 얼마나 되나요?
A: 일반 배송은 2-3일, 빠른 배송은 1일 소요됩니다.
   (신뢰도: 0.82)

Q: 환불하고 싶어요
A: 죄송합니다. 해당 질문에 대한 답변을 찾을 수 없습니다.
   (신뢰도: 0.00)

Q: 결제는 어떻게 해요?
A: 결제 시 포인트 사용 선택하시면 됩니다.
   (신뢰도: 0.50)

Q: 제품 AS 문의
A: 죄송합니다. 해당 질문에 대한 답변을 찾을 수 없습니다.
   (신뢰도: 0.00)


---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 단어 빈도 계산 ⭐

**문제**: 다음 리뷰 텍스트에서 가장 많이 등장한 단어와 그 빈도를 출력하세요.

```python
review = "배송이 빠르고 상품이 좋아요 배송도 빠르고 서비스도 좋아요"
```

**기대 결과**: `('좋아요', 2)` 또는 `('빠르고', 2)`

In [16]:
from collections import Counter

review = "배송이 빠르고 상품이 좋아요 배송도 빠르고 서비스도 좋아요"

# 여기에 코드를 작성하세요


### Q2. bigram 추출 ⭐

**문제**: 주어진 텍스트에서 모든 bigram(연속된 2단어)을 추출하세요.

```python
text = "파이썬 데이터 분석 입문"
```

**기대 결과**: `['파이썬 데이터', '데이터 분석', '분석 입문']`

In [ ]:
text = "파이썬 데이터 분석 입문"

# 여기에 코드를 작성하세요


### Q3. CountVectorizer 기초 ⭐⭐

**문제**: CountVectorizer를 사용하여 문서-단어 행렬을 만들고, '배송'이라는 단어가 각 문서에 몇 번 등장하는지 출력하세요.

```python
documents = [
    "배송이 빨라요 배송 최고",
    "상품이 좋아요",
    "배송도 빠르고 상품도 좋아요"
]
```

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

documents = [
    "배송이 빨라요 배송 최고",
    "상품이 좋아요",
    "배송도 빠르고 상품도 좋아요"
]

# 여기에 코드를 작성하세요


### Q4. TF-IDF 키워드 추출 ⭐⭐

**문제**: TfidfVectorizer를 사용하여 첫 번째 문서의 TF-IDF 값이 가장 높은 단어를 찾으세요.

```python
documents = [
    "삼성전자 반도체 수출",
    "현대자동차 전기차 판매",
    "LG전자 가전 수출"
]
```

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

documents = [
    "삼성전자 반도체 수출",
    "현대자동차 전기차 판매",
    "LG전자 가전 수출"
]

# 여기에 코드를 작성하세요


### Q5. 자카드 유사도 계산 ⭐⭐

**문제**: 두 문서의 자카드 유사도를 계산하세요.

```python
doc1 = "파이썬 데이터 분석"
doc2 = "파이썬 머신러닝 분석"
```

**힌트**: 교집합 / 합집합

In [ ]:
doc1 = "파이썬 데이터 분석"
doc2 = "파이썬 머신러닝 분석"

# 여기에 코드를 작성하세요


---

## Part 2 심화 퀴즈

---

### Q6. 코사인 유사도로 유사 문서 찾기 ⭐⭐⭐

**문제**: 기준 문서와 가장 유사한 문서를 찾고 유사도를 출력하세요.

```python
documents = [
    "파이썬 머신러닝 딥러닝",  # 기준 문서 (인덱스 0)
    "자바 웹 개발",
    "파이썬 데이터 분석 머신러닝",
    "자바스크립트 프론트엔드"
]
```

**기대 결과**: 문서 인덱스와 유사도 값

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

documents = [
    "파이썬 머신러닝 딥러닝",
    "자바 웹 개발",
    "파이썬 데이터 분석 머신러닝",
    "자바스크립트 프론트엔드"
]

# 여기에 코드를 작성하세요


### Q7. n-gram + TF-IDF 조합 ⭐⭐⭐

**문제**: unigram과 bigram을 모두 포함하는 TfidfVectorizer를 만들고, 첫 번째 문서에서 TF-IDF 값이 가장 높은 상위 3개 feature를 출력하세요.

```python
documents = [
    "배송이 빠르고 상품 품질 좋아요",
    "상품 품질은 좋은데 배송이 느려요",
    "가격 대비 만족스러워요"
]
```

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

documents = [
    "배송이 빠르고 상품 품질 좋아요",
    "상품 품질은 좋은데 배송이 느려요",
    "가격 대비 만족스러워요"
]

# 여기에 코드를 작성하세요


### Q8. 중복 문서 탐지 ⭐⭐⭐⭐

**문제**: 코사인 유사도가 0.6 이상인 문서 쌍을 모두 찾아 출력하세요.

```python
documents = [
    "삼성전자 반도체 수출 증가",
    "현대차 전기차 판매 호조",
    "삼성 반도체 수출 사상 최대",  # 0번과 유사
    "LG전자 가전 사업 성장",
    "삼성전자 반도체 수출 신기록"   # 0번과 유사
]
```

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

documents = [
    "삼성전자 반도체 수출 증가",
    "현대차 전기차 판매 호조",
    "삼성 반도체 수출 사상 최대",
    "LG전자 가전 사업 성장",
    "삼성전자 반도체 수출 신기록"
]

# 여기에 코드를 작성하세요


### Q9. 검색 쿼리 매칭 ⭐⭐⭐⭐

**문제**: 검색 쿼리 "데이터 분석"과 가장 관련 높은 문서 2개를 찾아 유사도와 함께 출력하세요.

```python
documents = [
    "파이썬 기초 문법 강좌",
    "데이터 분석을 위한 판다스",
    "웹 개발 입문 가이드",
    "머신러닝 데이터 전처리",
    "SQL 데이터베이스 기초"
]

query = "데이터 분석"
```

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

documents = [
    "파이썬 기초 문법 강좌",
    "데이터 분석을 위한 판다스",
    "웹 개발 입문 가이드",
    "머신러닝 데이터 전처리",
    "SQL 데이터베이스 기초"
]

query = "데이터 분석"

# 여기에 코드를 작성하세요


### Q10. 종합: 문서 분류 및 키워드 추출 시스템 ⭐⭐⭐⭐⭐

**문제**: 다음 요구사항을 모두 만족하는 분석 시스템을 구현하세요.

1. TF-IDF로 각 문서의 상위 2개 키워드 추출
2. 코사인 유사도로 가장 유사한 문서 쌍 찾기
3. Plotly로 문서 간 유사도 히트맵 시각화

```python
documents = [
    "삼성전자 스마트폰 갤럭시 신제품 출시",
    "애플 아이폰 신제품 발표",
    "현대차 전기차 아이오닉 판매 증가",
    "테슬라 전기차 모델Y 인기",
    "삼성 갤럭시 폴드 스마트폰 혁신"
]
```

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import plotly.figure_factory as ff
import pandas as pd

documents = [
    "삼성전자 스마트폰 갤럭시 신제품 출시",
    "애플 아이폰 신제품 발표",
    "현대차 전기차 아이오닉 판매 증가",
    "테슬라 전기차 모델Y 인기",
    "삼성 갤럭시 폴드 스마트폰 혁신"
]

# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 메소드/클래스 | 실무 활용 |
|------|------------------|----------|
| 단어 빈도 | `Counter`, `most_common()` | 키워드 순위, 불만 분석 |
| n-gram | 직접 구현 또는 `ngram_range` | 연속 표현 패턴 탐지 |
| CountVectorizer | `fit_transform()`, `get_feature_names_out()` | 문서-단어 행렬 생성 |
| TF-IDF | `TfidfVectorizer` | 핵심 키워드 추출, 중요도 측정 |

### Part 2: 심화 핵심 요약

| 기법 | 사용법 | 언제 쓰나? |
|------|--------|----------|
| 코사인 유사도 | `cosine_similarity(tfidf_matrix)` | 긴 문서 비교, 추천 시스템 |
| 자카드 유사도 | `len(A&B) / len(A|B)` | 짧은 텍스트, 태그 비교 |
| 중복 탐지 | 유사도 > 임계값 | 표절 검사, 스팸 필터링 |
| 문서 검색 | 쿼리 벡터 + 유사도 | FAQ 봇, 검색 엔진 |

### 💡 실무 팁

1. **전처리 중요**: TF-IDF 전에 불용어 제거, 형태소 분석 적용
2. **n-gram 범위**: bigram까지가 일반적, trigram은 데이터 많을 때만
3. **유사도 임계값**: 중복 탐지 시 0.7~0.8, 추천은 0.3~0.5
4. **희소 행렬**: 대용량 문서는 `sparse matrix` 유지 (toarray() 자제)
5. **한글 분석**: KoNLPy 형태소 분석과 조합하면 정확도 향상